In [4]:
from transformers import BertModel, BertTokenizer
import torch
from scipy import spatial
# Load pre-trained BERT model and tokenizer
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name).cuda()

# Tokenize the sentence
sentence1 = "I want to be a banker."
sentence2 = "banker is great"
model
def vector(sentence1, sentence2):
    tokens1 = tokenizer.encode(sentence1, add_special_tokens=True)
    tokens2 = tokenizer.encode(sentence2, add_special_tokens=True)
    # Generate the sentence vector
    input_ids1 = torch.tensor([tokens1]).cuda()
    input_ids2 = torch.tensor([tokens2]).cuda()
    with torch.no_grad():
        output = model(input_ids1)
        vector1 = output.last_hidden_state.mean(dim=1).squeeze().cpu()
        output = model(input_ids2)
        vector2 = output.last_hidden_state.mean(dim=1).squeeze().cpu()
    return 1 - spatial.distance.cosine(vector1, vector2)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [6]:
import pandas as pd
import numpy as np
job_df = pd.read_csv('job_with_skills.csv')
course_df = pd.read_csv('output_cleaned.csv')
course_df.dropna(subset=['Course_Hard_Skills'],inplace=True,axis=0)
course_df = course_df.reset_index(drop=True)
training_columns = ['Job','Softskills','Hardskills','Course_Name','Course_Description','Course_Hard_Skills','Course_Soft_Skills','label']
training_pd = pd.DataFrame(data=[[0, 1, 2, 3, 4, 5, 6, 7]], columns = training_columns)
for i in range(job_df.shape[0]):
    job_sen = job_df.loc[i,'Hardskills']
    scores = []
    for j in range(course_df.shape[0]):
        course_sen = course_df.loc[j,'Course_Hard_Skills']
        scores.append(vector(job_sen, course_sen))
    scores = np.array(scores)
    indexes = np.argsort(scores)[::-1]
    label_1 = indexes[:10]
    label_0 = np.random.choice(indexes[50:],20,replace = False)
    print(label_1,label_0)
    for idx in label_1:
        training_pd.loc[training_pd.shape[0],:]=list(job_df.loc[i,:])+list(course_df.loc[idx,['Course_Name','Course_Description','Course_Hard_Skills','Course_Soft_Skills']])+\
        [1]
    for idx in label_0:
        training_pd.loc[training_pd.shape[0],:]=list(job_df.loc[i,:])+list(course_df.loc[idx,['Course_Name','Course_Description','Course_Hard_Skills','Course_Soft_Skills']])+\
        [0]
training_pd = training_pd[1:].reset_index(drop=True)

[ 82  22  76 115  41 124  75 167  23 171] [ 55 137  93  65  74  68 112 173 119 131 116  84  43  64  42 143 180 100
  27  96]
[ 82  22  76 124 167  41 115  15  75  23] [146 163 139  42 105   5 144 131  73  94 159 168  55 104 100  62 152  72
  27  80]
[ 17  19   8  16  88  12  90 100   4   0] [ 56 107 191   1 141 144  29  32  50 156 166 138 174 171  26 135 158 124
  74 190]
[189 124  96  89  22  12 190 171 182 115] [ 92  44 101 103  62  25 137  30 165 108 105  35 136 130 161 179  38 119
  88 145]
[124 167   9 162 127  57  76  31 106  51] [173 120   3  84 153  71 144 179 166 112 129 118  44 134  73 191 161   2
 136  20]
[124  74 162  76  72 127 182 106 167  57] [ 25  36 138  38 156 153 139  19  97   6 102 188  60  46 155  18  86 118
 160   1]
[ 76  82  23  22 124 167 115  41  38  21] [150 113 154 118  54 177  62  27   4 125 144  48 120 158  44  67  10 110
   2 161]
[ 76 115 107 106   9 124  80 164 167 105] [176  66  81   3 152  75  37  71  54  49 135   8   1 175 168  59  23 120
  48  70]


In [7]:
training_pd

,Job,Softskills,Hardskills,Course_Name,Course_Description,Course_Hard_Skills,Course_Soft_Skills,label
0,Mortgage Advisor,"Communication skills, active listening, proble...","Good communication and listening skills, The a...",Investments and portfolio analysis,introduces students to the fundamental princip...,Familiarity with fundamental principles of inv...,Sophisticated assessment of current issues and...,1.0
1,Mortgage Advisor,"Communication skills, active listening, proble...","Good communication and listening skills, The a...",Impact lab,an opportunity to work directly for social ven...,ability to manage tasks and solve real-life pr...,working directly for social ventures under gui...,1.0
2,Mortgage Advisor,"Communication skills, active listening, proble...","Good communication and listening skills, The a...",Senior seminar in economics and finance,applies core theoretical knowledge in economic...,"research skills, theoretical knowledge in econ...",NaN,1.0
3,Mortgage Advisor,"Communication skills, active listening, proble...","Good communication and listening skills, The a...",Behavioural and sociological finance,covers current developments in finance. Possib...,"Understanding current developments in finance,...",NaN,1.0
4,Mortgage Advisor,"Communication skills, active listening, proble...","Good communication and listening skills, The a...",Theory of international trade,deals with economic issues which are related t...,knowledge of economic theory and analytical sk...,NaN,1.0
...,...,...,...,...,...,...,...,...
565,Industry Analyst,"Analytical skills, problem-solving, attention ...","Understanding the Business Objective, Analytic...",Chinese economy,provides a general introduction of Chinese eco...,knowledge of Chinese economic history since 19...,ability to understand the development and tran...,0.0
566,Industry Analyst,"Analytical skills, problem-solving, attention ...","Understanding the Business Objective, Analytic...","Creativity, innovation and entrepreneurship in...",provides students with a comprehensive study o...,"economic analysis, knowledge of innovation and...",NaN,0.0
567,Industry Analyst,"Analytical skills, problem-solving, attention ...","Understanding the Business Objective, Analytic...",Advanced topics in taxation,provides students with a thorough and in-depth...,"knowledge of current law, principles and pract...",NaN,0.0
568,Industry Analyst,"Analytical skills, problem-solving, attention ...","Understanding the Business Objective, Analytic...",The political economy of globalization,This course provides a theoretical framework t...,"theoretical analysis, economic analysis, case ...",NaN,0.0


In [9]:
training_pd.to_csv('training_pd_new.csv',index=False)